# LangGraph Advanced with Llama 3

Will apply [Llama 3 (8B Model)](https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3) through [Ollama](https://python.langchain.com/docs/integrations/chat/ollama/) to build [LangGraph](https://python.langchain.com/docs/langgraph/) multi-agent RAG Sytems

Ensure that you have `Ollama` running and have pulled `Llama3:8B` model 

In [8]:
import os
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import YoutubeLoader

Inspiration from [Langchain Videos](https://www.youtube.com/watch?v=-ROS6gfYIts) and [previous notebook](https://github.com/jzamalloa1/langchain_learning/blob/main/langgraph_testing.ipynb)

<p>
<img src="ILLUSTRATIONS/langgraph_advanced_flow.png" 
      width="65%" height="auto"
      style="display: block; margin: 0 auto" />

Illustration [reference](https://github.com/jzamalloa1/langchain_learning/blob/main/langgraph_testing.ipynb)

In [9]:
# OpenAI API Key solely to use embedding model
os.environ["OPENAI_API_KEY"] = ""
os.environ["TAVILY_API_KEY"] = ""

In [3]:
llama_model = "llama3:8b"

#### Source for Vector Stores

In [5]:
llm_urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
    
]

plotly_yt_urls = [
    "https://www.youtube.com/watch?v=Qx5eFVUdDxk&list=PLYD54mj9I2JevdabetHsJ3RLCeMyBNKYV&index=1",
    "https://www.youtube.com/watch?v=Z9YUejzkFa0&list=PLYD54mj9I2JevdabetHsJ3RLCeMyBNKYV&index=2",
    "https://www.youtube.com/watch?v=4bP66rRxVBw&list=PLYD54mj9I2JevdabetHsJ3RLCeMyBNKYV&index=3",
    "https://www.youtube.com/watch?v=a1qzu5GKIf0&list=PLYD54mj9I2JevdabetHsJ3RLCeMyBNKYV&index=4",
    "https://www.youtube.com/watch?v=Fm7DC-Z5R7A&list=PLYD54mj9I2JevdabetHsJ3RLCeMyBNKYV&index=5",
    "https://www.youtube.com/watch?v=4jcWJ30HqSY&list=PLYD54mj9I2JevdabetHsJ3RLCeMyBNKYV&index=6"
]

#### Build Retriever

In [ ]:
# Vector store for llm urls

llm_docs = [WebBaseLoader(url).load() for url in llm_urls] # Each loader produced a list of one element
llm_docs = [i for j in llm_docs for i in j] # Decoupling lists of one element

text_splitter_class = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)

chunks = text_splitter_class.split_documents(llm_docs)
print(f"Split into {len(chunks)} chunks")

llm_llw_vectorstore = Chroma.from_documents(
    documents = chunks,
    embedding = embedding_model,
    collection_name = "chroma_llm_llw"
)

llm_llw_retriever = llm_llw_vectorstore.as_retriever()